In [5]:
%matplotlib inline
import numpy as np
import scipy 
import cvxpy as cvx
import random
from scipy.stats import multivariate_normal as mn
from sklearn.linear_model import LogisticRegression as LR

## Setup the synthetic data

In [2]:
eta = 0.01
nC = 20 # number of points per class
p  = 3 # number of domains
C  = 10 # number of classes
n  = nC*p*C
U = 1.0/n

In [3]:
def get_random_D_h(nC, p, C):
    D = np.random.rand(nC*p*C,C,p) # prob point n is in domain p and class C
    h = np.random.rand(nC*p*C,C,p) # score for point n in domain p and class C
    # normalize D and h
    for k in range(p): # loop over domain
        D[:,:,k] = D[:,:,k] / D[:,:,k].sum()
    
        for i in range(nC):
            h[i,:,k] = h[i,:,k]/h[i,:,k].sum() # since this is the output of a softmax function its normalized over classes 
    return D,h

# Define a data problem

In [22]:
class synthetic_problem():
    
    def __init__(self, p=3, nC=20, c=10, seed=1337):
        self.p = p # number of domains
        self.nC = nC # pts per class
        self.C = C # number of classes
        self.n = nC * p * C # num pts
        self.U = 1.0 / self.n # unif dist
        self.eta = 0.01
        self.seed = seed
        
        np.random.seed(seed)
        self.generate_density()
        self.generate_regressor()
        
    def generate_density(self):
        D = np.random.rand(self.n,self.p)
        # Normalize each domain
        for k in range(self.p):
            D[:,k] = D[:,k] / D[:,k].sum()
        self.D = D
    
    def generate_regressor(self):
        h = np.random.rand(self.n,self.p)
        for k in range(self.p):
            h[:,k] = h[:,k] / h[:,k].sum()
        self.h = h
        
        # compute H
        self.H = np.zeros(self.n)
        for i in range(self.n):
            self.H[i] = 1.0 / self.p * h[i,:].sum()
        
    def get_marginal_density(self):
        return self.D
    
    def get_regressor(self):
        return self.h
    
    def get_H(self):
        return self.H

# Optimization

## Objective
* z_{t+1} = argmin gamma
* subject to
    * u_k(z) - v_k(z_t) - (z - z_t) grad_{v_k(z_t)} <= g for all k in [p]
    * -z_k <= 0 forall k in [p]
    * sum_{k=1}^p z_k - 1 = 0

In [193]:
def compute_H(x, DP):
    return DP.get_H()[x]

def compute_Dz(x, z, DP):
    """ Dz = sum_k z_k * D_k(x)"""
    D = DP.get_marginal_density()[x,:]
    Dz = 0
    for k in range(DP.p):
        Dz += z[k] * D[k]
    return Dz

def compute_Jz(x, z, DP):
    const = DP.eta * DP.U * compute_H(x, DP)
    D = DP.get_marginal_density()[x,:]
    h = DP.get_regressor()[x,:]
#    zDh = z*D*h
    zDh = 0
    for k in range(DP.p):
        zDh += z[k] * D[k] * h[k]
    return zDh + p*const
#    return sum(zDh + const)

def compute_Kz(x, z, DP):
    return compute_Dz(x, z, DP) + DP.eta * DP.U

def compute_hz(x, z, DP, Jz=None, Kz=None, CX=False):
    if Jz is None:
        Jz = compute_Jz(x, z, DP)
    if Kz is None:
        Kz = compute_Kz(x, z, DP)
        
    if CX:
        return Jz * cvx.inv_pos(Kz)
    else:
        return Jz / Kz

def compute_fz(x, z, DP, Jz=None, Kz=None):
    if not Jz:
        Jz = compute_Jz(x, z, DP)
    if not Kz:
        Kz = compute_Kz(x, z, DP)
    return (Jz + 1) ** 2 / (2*Kz)

def compute_gz(x, z, DP, Jz=None, Kz=None):
    if not Jz:
        Jz = compute_Jz(x, z, DP)
    if not Kz:
        Kz = compute_Kz(x, z, DP)
    return ((Jz**2) + 1) / (2*Kz)

def compute_Fz(x, z, DP, fz=None, gz=None):
    if not fz:
        fz = compute_fz(x, z, DP)
    if not gz:
        gz = compute_gz(x, z, DP)
    return 2 * (fz**2) + 2 * (gz**2)

def compute_Gz(x, z, DP, fz=None, gz=None):
    if not fz:
        fz = compute_fz(x, z, DP)
    if not gz:
        gz = compute_gz(x, z, DP)
    return (fz + gz)**2

def compute_grad_Jz(x, z, DP):
    D = DP.get_marginal_density()
    h = DP.get_regressor()
    return D[x,:] * h[x,:]

def compute_grad_Kz(x, z, DP):
    D = DP.get_marginal_density()[x,:]
    return D

def compute_grad_gz(x, z, DP, grad_Jz=None, 
                    grad_Kz=None, Jz=None,
                   Kz=None):
    if grad_Jz is None:
        grad_Jz = compute_grad_Jz(x, z, DP)
    if grad_Kz is None:
        grad_Kz = compute_grad_Kz(x, z, DP)
    if not Jz:
        Jz = compute_Jz(x, z, DP)
    if not Kz:
        Kz = compute_Kz(x, z, DP)
    return (Jz * grad_Jz) / Kz - (((Jz**2) + 1)*grad_Kz) / Kz

def compute_grad_fz(x, z, DP, Jz=None, Kz=None,
                    grad_Jz=None, grad_Kz=None):
    if grad_Jz is None:
        grad_Jz = compute_grad_Jz(x, z, DP)
    if grad_Kz is None:
        grad_Kz = compute_grad_Kz(x, z, DP)
    if not Jz:
        Jz = compute_Jz(x, z, DP)
    if not Kz:
        Kz = compute_Kz(x, z, DP)
    return (Jz + 1)*grad_Jz / Kz - (Jz+1)**2 * grad_Kz / (Kz**2)

def compute_grad_Gz(x, z, DP, fz=None, gz=None,
                   grad_fz=None, grad_gz=None):
    if not fz:
        fz = compute_fz(x, z, DP)
    if not gz:
        gz = compute_gz(x, z, DP)
    if grad_fz is None:
        grad_fz = compute_grad_fz(x, z, DP)
    if grad_gz is None:
        grad_gz = compute_grad_gz(x, z, DP)
    return 2 * (fz + gz) * (grad_fz + grad_gz)

In [185]:
DP = synthetic_problem()
zp = np.repeat(DP.U, DP.p)
x=0
print 'z:', zp, 'x:', x
print 'H(x)', compute_H(x, DP)
print 'Dz(x)', compute_Dz(x, zp, DP)
print 'Jz(x)', compute_Jz(x, zp, DP)
print 'Kz(x)', compute_Kz(x, zp, DP)
print 'fz(x)', compute_fz(x, zp, DP)
print 'gz(x)', compute_gz(x, zp, DP)
print 'Fz(x)', compute_Fz(x, zp, DP)
print 'Gz(x)', compute_Gz(x, zp, DP)
print 'grad_Jz(x)', compute_grad_Jz(x, zp, DP)
print 'grad_Kz(x)', compute_grad_Kz(x, zp, DP)
print 'grad_gz(x)', compute_grad_gz(x, zp, DP)
print 'grad_fz(x)', compute_grad_fz(x, zp, DP)
print 'grad_Gz(x)', compute_grad_Gz(x, zp, DP)

z: [ 0.00166667  0.00166667  0.00166667] x: 0
H(x) 0.00204966305016
Dz(x) 3.82396517581e-06
Jz(x) 1.10664488896e-07
Kz(x) 2.04906318425e-05
fz(x) 24401.400333
gz(x) 24401.3949323
Fz(x) 2381712825.7
Gz(x) 2381712825.7
grad_Jz(x) [  1.27558605e-06   7.93751866e-07   2.83946392e-06]
grad_Kz(x) [ 0.00087519  0.00053069  0.0008885 ]
grad_gz(x) [-42.71173161 -25.898915   -43.36145472]
grad_fz(x) [-2084452.06200754 -1263939.54750309 -2116160.29349099]
grad_Gz(x) [ -2.03458343e+11  -1.23370094e+11  -2.06553307e+11]


In [186]:
print 'z:', zp, 'x:', x
H = compute_H(x, DP)
Dz = compute_Dz(x, zp, DP)
Jz = compute_Jz(x, zp, DP)
Kz = compute_Kz(x, zp, DP)
fz = compute_fz(x, zp, DP, Jz=Jz, Kz=Kz)
gz = compute_gz(x, zp, DP, Jz=Jz, Kz=Kz)
Fz = compute_Fz(x, zp, DP, fz=fz, gz=gz)
Gz = compute_Gz(x, zp, DP, fz=fz, gz=gz)
grad_Jz = compute_grad_Jz(x, zp, DP)
grad_Kz = compute_grad_Kz(x, zp, DP)
grad_gz = compute_grad_gz(x, zp, DP, Jz=Jz, 
                          Kz=Kz, grad_Jz=grad_Jz,
                         grad_Kz=grad_Kz)
print 'fz(x)', fz
print 'gz(x)', gz
print 'Fz(x)', Fz
print 'Gz(x)', Gz
print 'grad_Jz(x)', grad_Jz
print 'grad_Kz(x)', grad_Kz
print 'grad_gz(x)', grad_gz
print 'grad_fz(x)', compute_grad_fz(x, zp, DP)
print 'grad_Gz(x)', compute_grad_Gz(x, zp, DP)

z: [ 0.00166667  0.00166667  0.00166667] x: 0
fz(x) 24401.400333
gz(x) 24401.3949323
Fz(x) 2381712825.7
Gz(x) 2381712825.7
grad_Jz(x) [  1.27558605e-06   7.93751866e-07   2.83946392e-06]
grad_Kz(x) [ 0.00087519  0.00053069  0.0008885 ]
grad_gz(x) [-42.71173161 -25.898915   -43.36145472]
grad_fz(x) [-2084452.06200754 -1263939.54750309 -2116160.29349099]
grad_Gz(x) [ -2.03458343e+11  -1.23370094e+11  -2.06553307e+11]


In [194]:
def compute_u(z, DP):
    D = DP.get_marginal_density()
    h = DP.get_regressor()
    etaU = DP.eta * DP.U

    # sum_k z_k * (sum_x Dk(x) * h_k(x)^2)
    const = 0
    v = (D * (h**2)).sum(axis=0)
    for k in range(DP.p):
        const += z[k] * v[k]
    
    u = np.zeros(DP.p)
    for x in range(DP.n):
        H = compute_H(x, DP)
        Dz = compute_Dz(x, z, DP)
        Jz = compute_Jz(x, z, DP)
        Kz = compute_Kz(x, z, DP)
        hz = compute_hz(x, z, DP, Jz=Jz, Kz=Kz, CX=True)
        fz = compute_fz(x, z, DP, Jz=Jz, Kz=Kz)
        gz = compute_gz(x, z, DP, Jz=Jz, Kz=Kz)
        Fz = compute_Fz(x, z, DP, fz=fz, gz=gz)
        Gz = compute_Gz(x, z, DP, fz=fz, gz=gz)
        for k in range(DP.p):
            Dk = D[x,k]
            hk = h[x,k]

            v1 = Dk * (Fz + 2*hk*gz + hk**2)
            v2 = etaU*Fz + Jz*hz + 2*etaU*H*gz
            u[k] += v1 + v2 - const
    return u
        

def compute_v(z, DP):
    D = DP.get_marginal_density()
    h = DP.get_regressor()
    H = DP.get_H()
    
    v = np.zeros(DP.p)
    for x in range(DP.n):
        Gz = compute_Gz(x, z, DP)
        fz = compute_fz(x, z, DP)
        etaU = DP.eta * DP.U
        etaUH = etaU * H[x]
        va = D[x,:] + etaU
        vb = 2 * h[x,:]*D[x,:] + 2 * etaUH
        v += (Gz * va) + (fz * vb)

    return v
        

def compute_grad_v(z, DP):
    D = DP.get_marginal_density()
    h = DP.get_regressor()
    H = DP.get_H()
    
    grad_v = np.zeros(DP.p)
    for x in range(DP.n):
        grad_Gz = compute_grad_Gz(x, z, DP)
        grad_fz = compute_grad_fz(x, z, DP)
        etaU = DP.eta * DP.U
        etaUH = etaU * H[x]
        va = D[x,:] + etaU
        vb = 2 * h[x,:]*D[x,:] + 2 * etaUH
        
        grad_v += (grad_Gz * va) + (grad_fz * vb)

    return grad_v    

In [196]:
u = compute_u(zp, DP)
print u[:,np.newaxis].shape

ValueError: setting an array element with a sequence.

# Optimization

In [189]:
def solve_iter(z_prev, DP):
    # setup variables
    z = cvx.Variable(DP.p) # variables are z
    print z.size, z_prev.shape, type(z), type(z_prev)
    gamma = cvx.Variable()
    objective = cvx.Minimize(gamma)

    constraints = [0 <= z, cvx.sum_entries(z) == 1]
    grad_v = compute_grad_v(z_prev, DP)
    v = compute_v(z_prev, DP)
    u = compute_u(z, DP)
    constraints.append(u - z*grad_v + (v - z_prev * grad_v))
    #"""Stopped here."""
    #return
    #
    #for k in range(p):
    #    g_ = compute_grad_vk(k, z_prev, DP)
    #    v_k = compute_vk(k, z_prev, DP)
    #    u_k_z = compute_uk_z(k, z, DP)
    #    c_k = - cvx.sum_entries(cvx.mul_elemwise(g_,z)) +  sum(z_prev*g_)
    #    constraints.append(u_k_z - v_k + c_k <= gamma )
    #print 'setup problem'
    prob = cvx.Problem(objective, constraints)

    #solvers.options['feastol']=1e-6      # slightly relaxed from default (avoids singular KKT messages)
    #solvers.options['abstol']= 1e-9      # gives us good accuracy on final result
    val = prob.solve(solver=cvx.ECOS, verbose=True, reltol=1e-4)
    out = np.array(z.value).reshape(p)
    print "Optimal value", val
    print "Optimal var", out # A numpy array.
    return out,val

In [190]:
print DP.get_marginal_density()[x,:].shape

(3,)


In [191]:
out, val = solve_iter(zp, DP)

(3, 1) (3,) <class 'cvxpy.expressions.variables.variable.Variable'> <type 'numpy.ndarray'>


DCPError: Can only divide by a scalar constant.

In [149]:
type(zp)

numpy.ndarray

In [153]:
tz = cvx.Variable(DP.p)

In [159]:
tz.size

(3, 1)

In [166]:
tz.size

(3, 1)